In [ ]:
import cv2
import os
import shutil
from PIL import Image
import numpy as np
import random

def apply_mask_to_bounding_box(input_dir, output_dir):
    """
    이미지와 바운딩 박스를 참조하여 바운딩 박스 영역의 50%를 랜덤 위치로 마스크 처리하고 결과를 저장.

    Parameters:
        input_dir (str): 원본 디렉토리 경로.
        output_dir (str): 마스크 처리된 이미지를 저장할 디렉토리 경로.
    """
    if not os.path.exists(input_dir):
        print(f"입력 디렉토리가 존재하지 않습니다: {input_dir}")
        return

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        output_subdir = os.path.join(output_dir, relative_path)

        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for file in files:
            input_file_path = os.path.join(root, file)
            output_file_path = os.path.join(output_subdir, file)

            # 이미지 파일 여부 확인
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                try:
                    # 이미지 로드
                    image = cv2.imread(input_file_path)
                    if image is None:
                        print(f"이미지를 로드할 수 없습니다: {input_file_path}")
                        continue

                    # 대응하는 바운딩 박스 파일(.txt) 확인
                    relative_txt_path = relative_path.replace('images', 'labels')
                    txt_file_path = os.path.join(input_dir.replace('images', 'labels'), relative_txt_path, os.path.splitext(file)[0] + '.txt')

                    if os.path.exists(txt_file_path):
                        with open(txt_file_path, 'r') as f:
                            lines = f.readlines()

                        for line in lines:
                            parts = line.strip().split()
                            if len(parts) < 5:
                                continue

                            # YOLO 포맷: class, x_center, y_center, width, height
                            _, x_center, y_center, width, height = map(float, parts)

                            # 이미지 크기에 맞게 바운딩 박스 좌표 변환
                            h, w, _ = image.shape
                            x_center *= w
                            y_center *= h
                            width *= w
                            height *= h

                            x1 = int(x_center - width / 2)
                            y1 = int(y_center - height / 2)
                            x2 = int(x_center + width / 2)
                            y2 = int(y_center + height / 2)

                            # 마스크 영역 계산 (50% 랜덤 위치)
                            mask_width = int(0.5 * width)
                            mask_height = int(0.5 * height)

                            rand_x1 = random.randint(x1, x2 - mask_width)
                            rand_y1 = random.randint(y1, y2 - mask_height)
                            rand_x2 = rand_x1 + mask_width
                            rand_y2 = rand_y1 + mask_height

                            # 바운딩 박스 내 랜덤 50% 영역 마스킹 처리
                            image[rand_y1:rand_y2, rand_x1:rand_x2] = 0  # 검은색으로 마스크 처리

                    # 마스크 처리된 이미지 저장
                    cv2.imwrite(output_file_path, image)
                except Exception as e:
                    print(f"이미지 처리 실패: {input_file_path}, 오류: {e}")
            else:
                # 이미지가 아닌 경우 파일 복사
                shutil.copy2(input_file_path, output_file_path)

# 사용 예시
input_directory = "/home/K2023511/dataset/aihub-wild-animal/validation"
output_directory = "/home/K2023511/dataset/aihub-wild-animal/validation_masked"
# input_directory = "/home/K2023511/korea_wildlife_object_detection/dataset/sample/val"
# output_directory = "/home/K2023511/korea_wildlife_object_detection/dataset/sample/val_masked"
apply_mask_to_bounding_box(input_directory, output_directory)
